[src](https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-1-fd544fab149)

### Policy Gradient
* policy loss eqn $- log(\pi) * A$
* A is advantage, and is an essential aspect of all reinforcement learning algorithms. Intuitively it corresponds to how much better an action was than some baseline. In an env where -1 and 1 50/50, the baseline is 0
* $\pi$ is the weight we gave to the chosen action

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
#List out our bandits. Currently bandit 4 (index#3) is set to most often provide a positive reward.
bandits = [0.2,0,-0.2,-5]
num_bandits = len(bandits)
def pullBandit(bandit):
    #Get a random number.
    result = np.random.randn(1)
    if result > bandit:
        #return a positive reward.
        return 1
    else:
        #return a negative reward.
        return -1

In [8]:
tf.reset_default_graph()

#These two lines established the feed-forward part of the network. This does the actual choosing.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

#The next six lines establish the training proceedure. We feed the reward and chosen action into the network
#to compute the loss, and use it to update the network.
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [39]:
total_episodes = 1000 #Set total number of episodes to train agent on.
total_reward = np.zeros(num_bandits) #Set scoreboard for bandits to 0.
e = 0.1 #Set the chance of taking a random action.

init = tf.initialize_all_variables()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        #Choose either a random action or one from our network.
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)

        reward = pullBandit(bandits[action]) #Get our reward from picking one of the bandits.

        #Update the network.
        _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #Update our running tally of scores.
        total_reward[action] += reward
        if i % 50 == 0:
            print "Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward)
        i+=1

print "The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising...."
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print "...and it was right!"
else:
    print "...and it was wrong!"

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Running reward for the 4 bandits: [-1.  0.  0.  0.]
Running reward for the 4 bandits: [  0.   1.   2.  36.]
Running reward for the 4 bandits: [  1.   2.   0.  82.]
Running reward for the 4 bandits: [   1.    0.    1.  129.]
Running reward for the 4 bandits: [   2.   -2.    1.  172.]
Running reward for the 4 bandits: [   1.   -1.    1.  218.]
Running reward for the 4 bandits: [   2.   -1.    1.  265.]
Running reward for the 4 bandits: [   1.   -2.    0.  310.]
Running reward for the 4 bandits: [   0.   -1.    1.  357.]
Running reward for the 4 bandits: [   0.   -1.    0.  406.]
Running reward for the 4 bandits: [  -3.   -2.    0.  448.]
Running reward for the 4 bandits: [  -4.   -4.    3.  492.]
Running reward for the 4 bandits: [  -4.   -4.    3.  538.]
Running reward for the 4 bandits: [  -5.   -5.    2.  581.]
Running reward for the 4 bandits: [  -4.   -7.    3.  627.]
Running reward for the 4 bandits: [  -3.  